In [ ]:
import random
import time
import pandas as pd
import io
import json
import zstandard as zstd

In [ ]:
# ==========================================
# CONFIGURATION
# ==========================================
# 1. Define your ORIGINAL large source files
path_to_folder = '../RawData/'
file_bridge    = '_raw_yt_metadata.jsonl.zst'   # The Master/Anchor file (JSONL Zstandard)
file_channels  = '_raw_df_channels.tsv.gz'     # Left Side
file_time      = '_raw_df_timeseries.tsv.gz'   # Left Side
file_num_comm  = 'num_comments.tsv.gz'         # Right Side
file_yt_comm   = 'youtube_comments.tsv.gz'     # Right Side

# 2. Output settings
target_sample_size = 100_000
out_prefix = '../SampleData/anchor_'

start_time = time.time()

In [ ]:
# ==========================================
# STEP 1: CREATE THE ANCHOR (THE BRIDGE)
# ==========================================
print(f"1. Anchoring on {path_to_folder + file_bridge}...")

# For JSONL.ZST: stream-decompress and parse line-by-line, collecting a candidate pool
bridge_rows = []  # list of dicts

# Stream read the ZST JSONL; build a candidate pool by fractional sampling per chunk
# We'll read in manageable text buffers; within each buffer, sample a fraction to keep memory bounded.
pool_target_multiplier = 4  # stop when we have ~4x target in the pool
fraction_per_chunk = 0.1    # similar strategy to previous chunk sampling

# Helper to sample fraction from a list without converting to DataFrame yet
def sample_fraction(records, frac, rng=random.Random(42)):
    if not records:
        return []
    # Convert fraction to count
    k = max(1, int(len(records) * frac))
    # If k >= len(records), just return the records
    if k >= len(records):
        return records
    return rng.sample(records, k)

print(f"   -> Streaming and sampling ~{int(fraction_per_chunk*100)}% of each chunk into pool...")

# Read the .jsonl.zst file
chunk_lines_target = target_sample_size * 2  # logical chunk size for pooling
current_pool_size = 0
rng = random.Random(42)

with open(path_to_folder + file_bridge, 'rb') as fh:
    dctx = zstd.ZstdDecompressor()
    with dctx.stream_reader(fh) as reader:
        text_stream = io.TextIOWrapper(reader, encoding='utf-8')
        chunk_buffer = []
        for i, line in enumerate(text_stream):
            try:
                rec = json.loads(line)
            except json.JSONDecodeError:
                continue  # skip malformed lines safely
            chunk_buffer.append(rec)

            # When buffer reaches target, sample a fraction and add to pool
            if len(chunk_buffer) >= chunk_lines_target:
                sampled = sample_fraction(chunk_buffer, fraction_per_chunk, rng)
                bridge_rows.extend(sampled)
                current_pool_size = len(bridge_rows)
                # Clear buffer
                chunk_buffer.clear()
                # Safety stop when pool large enough
                if current_pool_size > target_sample_size * pool_target_multiplier:
                    break
        # Handle any remaining buffer
        if chunk_buffer:
            sampled = sample_fraction(chunk_buffer, fraction_per_chunk, rng)
            bridge_rows.extend(sampled)

print(f"   -> Candidate pool size: {len(bridge_rows):,}")

# Combine candidates and take the exact 'target_sample_size'
# Convert to DataFrame for downstream processing
if not bridge_rows:
    raise RuntimeError("No records were collected from the bridge file. Check input path/format.")

df_bridge_pool = pd.DataFrame(bridge_rows)
if len(df_bridge_pool) < target_sample_size:
    raise RuntimeError(f"Pool smaller than target_sample_size: {len(df_bridge_pool)} < {target_sample_size}")

df_final_bridge = df_bridge_pool.sample(n=target_sample_size, random_state=42)

# Save the Master Bridge File as TSV.GZ to match downstream reading
# Ensure required keys exist
required_cols = []
if 'channel_id' in df_final_bridge.columns:
    required_cols.append('channel_id')
else:
    raise KeyError("'channel_id' not found in bridge records.")

# display_id in yt metadata maps to video_id on some files; verify presence
if 'display_id' in df_final_bridge.columns:
    required_cols.append('display_id')
else:
    # Sometimes the field might be named 'video_id' directly
    if 'video_id' in df_final_bridge.columns:
        df_final_bridge['display_id'] = df_final_bridge['video_id']
        required_cols.append('display_id')
    else:
        raise KeyError("Neither 'display_id' nor 'video_id' found in bridge records.")

# Persist full sampled bridge, not just required columns
output_path = (out_prefix + file_bridge.replace('.jsonl.zst', '.csv')).replace('__', '_')
df_final_bridge.to_csv(output_path, index=False)
print(f"   -> Saved anchor file with {len(df_final_bridge)} rows.")

# Extract the keys we need to filter other files
valid_channels = set(df_final_bridge['channel_id'].unique())
valid_videos   = set(df_final_bridge['display_id'].unique()) # 'display_id' maps to 'video_id' later

print(f"   -> Keys loaded: {len(valid_channels)} Channels, {len(valid_videos)} Videos.")

# ==========================================
# STEP 2: FILTER LEFT SIDE (Channel Based)
# ==========================================
# Maps filename to the column name inside it
left_files_map = {
    file_channels: 'channel',
    file_time:     'channel'
}

for fname, col_name in left_files_map.items():
    print(f"2. Processing {path_to_folder + fname}...")
    
    out_name = f"{out_prefix + fname}"
    first_chunk = True
    
    with pd.read_csv(path_to_folder + fname, sep='\t', compression='gzip', chunksize=50000) as reader:
        for chunk in reader:
            # Keep row ONLY if the channel existed in our Bridge sample
            if col_name not in chunk.columns:
                continue
            mask = chunk[col_name].isin(valid_channels)
            filtered_chunk = chunk[mask]
            
            if not filtered_chunk.empty:
                mode = 'w' if first_chunk else 'a'
                header = first_chunk
                filtered_chunk.to_csv(out_name.replace('.tsv.gz', '.csv').replace('__', '_'), index=False, mode=mode, header=header)
                first_chunk = False

# ==========================================
# STEP 3: FILTER RIGHT SIDE (Video Based)
# ==========================================
right_files_map = {
    file_num_comm: 'display_id',
    file_yt_comm:  'video_id'    # WATCH OUT: This file uses 'video_id'
}

for fname, col_name in right_files_map.items():
    print(f"3. Processing {path_to_folder + fname}...")
    
    out_name = f"{out_prefix + fname}"
    first_chunk = True
    
    with pd.read_csv(path_to_folder + fname, sep='\t', compression='gzip', chunksize=50000) as reader:
        for chunk in reader:
            # Keep row ONLY if the video ID existed in our Bridge sample
            if col_name not in chunk.columns:
                continue
            if col_name == 'video_id':
                mask = chunk[col_name].isin(valid_videos)
            else:  # 'display_id' case
                mask = chunk[col_name].isin(valid_videos)
            filtered_chunk = chunk[mask]
            
            if not filtered_chunk.empty:
                mode = 'w' if first_chunk else 'a'
                header = first_chunk
                filtered_chunk.to_csv(out_name.replace('.tsv.gz', '.csv').replace('__', '_'), index=False, mode=mode, header=header)
                first_chunk = False

# ==========================================
end_time = time.time()
print("="*30)
print("ANCHOR SAMPLING COMPLETE!")
print(f"Total processing time: {end_time - start_time:.2f} seconds.")

1. Anchoring on ../RawData/_raw_yt_metadata.jsonl.zst...
   -> Streaming and sampling ~10% of each chunk into pool...
   -> Candidate pool size: 420,000
   -> Candidate pool size: 420,000
   -> Saved anchor file with 100000 rows.
   -> Keys loaded: 6857 Channels, 99995 Videos.
   -> Saved anchor file with 100000 rows.
   -> Keys loaded: 6857 Channels, 99995 Videos.
